In [65]:
from glob import glob
import pandas as pd
import numpy as np
from scipy import stats  
import matplotlib.pyplot as plt
import japanize_matplotlib

In [66]:
files = glob("../csv/演習問題_データアナリティクス基礎/*")
for i, f in enumerate(files):
    print(i, f)


0 ../csv/演習問題_データアナリティクス基礎\第10章演習問題_乱数で発生させて分布の基本統計量.xlsx
1 ../csv/演習問題_データアナリティクス基礎\第2章演習問題_クレームデータ.xlsx
2 ../csv/演習問題_データアナリティクス基礎\第3章演習問題_職場ストレス.xlsx
3 ../csv/演習問題_データアナリティクス基礎\第4章演習問題_児童のネット利用時間.xlsx
4 ../csv/演習問題_データアナリティクス基礎\第5章演習問題_生鮮野菜の購入額.xlsx
5 ../csv/演習問題_データアナリティクス基礎\第7章演習問題_授業効果の検証.xlsx
6 ../csv/演習問題_データアナリティクス基礎\第8章演習問題1_健康食品・化粧品販売額.xlsx
7 ../csv/演習問題_データアナリティクス基礎\第8章演習問題2_百貨店の衣料品売上高.xlsx
8 ../csv/演習問題_データアナリティクス基礎\第9章演習問題_自由記述データ＆形態素解析データ.xlsx


クレームデータの分析

In [67]:
df = pd.read_excel(files[1])
print(df.columns)
df.columns = ['id', 'area', 'gender', 'contents']
df.info()
df.head()

Index(['データ番号', '販売地域', '苦情者性別', '苦情内容'], dtype='object')
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2890 entries, 0 to 2889
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        2890 non-null   int64 
 1   area      2890 non-null   object
 2   gender    2890 non-null   object
 3   contents  2890 non-null   object
dtypes: int64(1), object(3)
memory usage: 90.4+ KB


,id,area,gender,contents
0,1,Gエリア,女性,不要な契約強要
1,2,Gエリア,男性,料金割引の説明不足
2,3,Gエリア,女性,料金請求の不一致
3,4,Cエリア,女性,不要な契約強要
4,5,Gエリア,女性,その他


苦情内容の種類 : 8

In [68]:
df['contents'].unique()
print(f"Count of Complaint = {df['contents'].nunique()}")

Count of Complaint = 8


件数が最も多い苦情内容 : 料金請求の不一致 659件

In [69]:
contents = df.groupby('contents').count()['id'].sort_values(ascending=False)
contents

contents
料金請求の不一致     659
契約解除手続き      491
信用懸念         392
料金割引の説明不足    383
不要な契約強要      308
解釈条件の説明不足    269
営業員の態度       235
その他          153
Name: id, dtype: int64

上位3位までの割合: 53.4%

In [70]:
print(f'{contents[0:3].sum() / contents.sum():.01%}')

53.4%


Fエリアで割合の大きい苦情内容: 料金請求の不一致 0.21%

In [71]:
(df.query('area == "Fエリア"').groupby('contents').count()['id'] / df.query('area == "Fエリア"').count()[0]).sort_values(ascending=False)

contents
料金請求の不一致     0.205556
料金割引の説明不足    0.172222
解釈条件の説明不足    0.133333
信用懸念         0.127778
契約解除手続き      0.122222
営業員の態度       0.094444
不要な契約強要      0.083333
その他          0.061111
Name: id, dtype: float64

エリアと苦情内容の特化係数: Fエリアの最大特化係数 解釈条件の説明不足 0.089219

In [72]:
df_cross = df.pivot_table(index='area', columns='contents', values='id', aggfunc='count')
df_cross

contents,その他,不要な契約強要,信用懸念,営業員の態度,契約解除手続き,料金割引の説明不足,料金請求の不一致,解釈条件の説明不足
area,,,,,,,,
Aエリア,14,31,42,20,52,33,70,29
Bエリア,10,12,29,18,37,20,34,18
Cエリア,17,48,73,31,82,58,102,31
Dエリア,29,42,52,46,74,67,115,30
Eエリア,17,34,34,14,44,31,54,21
Fエリア,11,15,23,17,22,31,37,24
Gエリア,55,126,139,89,180,143,247,116


In [73]:
special_coefficient = pd.DataFrame()
for c in df_cross.columns:
    df_tmp = df_cross.loc[:, c] / df_cross.loc[:, c].sum()
    special_coefficient = pd.concat([special_coefficient, df_tmp], axis=1)
special_coefficient

,その他,不要な契約強要,信用懸念,営業員の態度,契約解除手続き,料金割引の説明不足,料金請求の不一致,解釈条件の説明不足
Aエリア,0.091503,0.100649,0.107143,0.085106,0.105906,0.086162,0.106222,0.107807
Bエリア,0.065359,0.038961,0.073980,0.076596,0.075356,0.052219,0.051593,0.066914
Cエリア,0.111111,0.155844,0.186224,0.131915,0.167006,0.151436,0.154780,0.115242
Dエリア,0.189542,0.136364,0.132653,0.195745,0.150713,0.174935,0.174507,0.111524
Eエリア,0.111111,0.110390,0.086735,0.059574,0.089613,0.080940,0.081942,0.078067
Fエリア,0.071895,0.048701,0.058673,0.072340,0.044807,0.080940,0.056146,0.089219
Gエリア,0.359477,0.409091,0.354592,0.378723,0.366599,0.373368,0.374810,0.431227
